## Set Up Dependencies and Data


In [ ]:
import joblib
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from teeplot import teeplot as tp


In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/chaynes2019/AvidaGeneDupe/abfd1563f20a0fa60229e68a125d9c3f6780493f/experiments/2022-3-29-PaperDuplication/TidiedData/TaskCountTimeCourse.csv")


## Reproducibility


In [ ]:
%load_ext watermark
%watermark -iwbmuvg -iv


In [ ]:
joblib.hash(df)


# Describe data


In [ ]:
df.head()


In [ ]:
df.describe()


In [ ]:
df.nunique()


In [ ]:
df["Treatment"].unique()


In [ ]:
df.groupby(["Treatment", "Replicate #"]).max("Time")["Time"].unique()


## Plot Data


In [ ]:
signif = "$^*$"
env = "static"


In [ ]:
def order(signif: str="") -> list:
    return [
        "Baseline",
        "High mutation rate",
        "Slip-scatter",
        "Slip-NOP",
        "Slip-random",
        f"Slip-scramble{signif}",
        f"Slip-duplicate{signif}",
    ]


In [ ]:
df_ = df.groupby(
    ["Treatment", "Replicate #"],
).max("Time").reset_index()
df_["Treatment"] = df_[
    "Treatment"
].replace(
    "Baseline-Treatment", "Baseline"
).replace(
    "High-Mutation", "High mutation rate"
).replace(
    "Slip-scramble", f"Slip-scramble{signif}"
).replace(
    "Slip-duplicate", f"Slip-duplicate{signif}",
)

with tp.teed(
    sns.catplot,
    data=df_,
    x="Treatment",
    y="Tasks Present",
    hue="Treatment",
    order=order(signif),
    hue_order=order(signif),
    aspect=1.4,
    height=2.5,
    inner="box",
    kind="violin",
    linewidth=1,
    palette="muted",
    teeplot_outattrs={"env": env},
) as g:
    ax, = g.axes.flat
    ax.set_ylabel("Phenotypic\nAdaptation Score")
    ax.set_xlabel("")
    ax.set_ylim(0, 9)
    ax.set_yticks(np.arange(0, 10, 1))
    plt.xticks(rotation=30, ha="right")


In [ ]:
df_ = df.copy()
df_["Treatment"] = df_[
    "Treatment"
].replace(
    "Baseline-Treatment", "Baseline"
).replace(
    "High-Mutation", "High mutation\nrate"
).replace(
    "Slip-scramble", f"Slip-scramble"
).replace(
    "Slip-duplicate", f"Slip-duplicate",
)

with tp.teed(
    sns.relplot,
    data=df_[
        df_["Time"] % 100 == 0  # speed up confidence interval bootstraps
    ],
    x="Time",
    y="Tasks Present",
    hue="Treatment",
    hue_order=order(""),
    aspect=0.9,
    errorbar="ci",
    height=2.5,
    kind="line",
    legend=False,
    linewidth=1,
    palette="muted",
    teeplot_outattrs={"env": env},
) as g:
    ax, = g.axes.flat
    ax.set_ylabel("Phenotypic Adaptation Score")
    ax.set_xlabel("Time (updates)")
    ax.set_ylim(0, 9)
    ax.set_xlim(0, None)
    ax.set_yticks(np.arange(0, 10, 1))
    plt.xticks(rotation=30, ha="right")
